In [1]:
#IMPORTAR LIBRERIAS PARA MANEJO DE LOS DATASETS
import pandas as pd

In [2]:
#LEVANTAR csv´s DE TRABAJO (plataformas)
amazon = pd.read_csv('amazon.csv')
disney = pd.read_csv('disney.csv')
hulu = pd.read_csv('hulu.csv')
netflix = pd.read_csv('netflix.csv')

In [2]:
#LEVANTAR csv's DE TRABAJO (score)
score1 = pd.read_csv('1.csv')
score2 = pd.read_csv('2.csv')
score3 = pd.read_csv('3.csv')
score4 = pd.read_csv('4.csv')
score5 = pd.read_csv('5.csv')
score6 = pd.read_csv('6.csv')
score7 = pd.read_csv('7.csv')
score8 = pd.read_csv('8.csv')

In [3]:
#ELIMINAR ALGUNOS DUPLICADOS DE LOS ARCHIVOS DE CALIFICACIÓN (124/11MM - 0.00113%)
score1.drop_duplicates(inplace=True)
score2.drop_duplicates(inplace=True)
score3.drop_duplicates(inplace=True)
score4.drop_duplicates(inplace=True)
score5.drop_duplicates(inplace=True)
score6.drop_duplicates(inplace=True)
score7.drop_duplicates(inplace=True)
score8.drop_duplicates(inplace=True)

In [8]:
#CREACION COLUMNA ID (LETRA INICIAL PLATAFORMA + show_id)
amazon["id"] = "a" + amazon["show_id"]
disney["id"] = "d" + disney["show_id"]
hulu["id"] = "h" + hulu["show_id"]
netflix["id"] = "n" + netflix["show_id"]

In [9]:
#ELIMINAR COLUMNA show_id DATASETS PLATAFORMAS
amazon.drop(columns=["show_id"], inplace=True)
disney.drop(columns=["show_id"], inplace=True)
hulu.drop(columns=["show_id"], inplace=True)
netflix.drop(columns=["show_id"], inplace=True)

In [10]:
#UNIR DATASETS DE LAS PLATAFORMAS
platall = pd.concat([amazon, disney, hulu, netflix], ignore_index=True)

In [11]:
#CAMBIAR NOMBRE A COLUMNA id PARA RELACIONAR CON DATASETS DE VALORACIONES USUARIO
platall.rename(columns={'id':'movieId'}, inplace=True)

In [4]:
#UNIR DATASETS DE LAS VALORACIONES
scoring = pd.concat([score1, score2, score3, score4, score5, score6, score7, score8], ignore_index=True)

In [5]:
scoring.to_csv("scoring.csv", sep=';')

In [6]:
scoring.shape

(11024165, 4)

In [13]:
#ELIMINAR COLUMNAS user_id y timestamp (NO NECESARIAS PARA LA SECCIÓN DE FUNCIONALIDADES API)
scoring.drop(columns=['userId', 'timestamp'], inplace=True)

In [14]:
#AGRUPAR DIFERENTES TIPO DE SHOW POR CALIFICACIÓN PROMEDIO
gen_score = scoring.groupby(['movieId']).mean()

In [15]:
#UNIR DATASETS DE PLATAFORMAS Y VALORACIONES
platforms = pd.merge(platall, gen_score, on=["movieId"])

In [17]:
#RENOMBRAR COLUMNAS
platforms.rename(columns={'movieId':'show_id'}, inplace=True)
platforms.rename(columns={'rating_x':'class'}, inplace=True)
platforms.rename(columns={'rating_y':'score'}, inplace=True)

In [18]:
#REEMPLAZAR VALORES NULOS DEL CAMPO class POR STRING 'G'
platforms['class'].fillna("G", inplace=True)

In [19]:
#CAMBIAR LAS FECHAS AL FORMATO AAAA-mm-dd
platforms['date_added'] = pd.to_datetime(platforms['date_added'])

In [20]:
#LLEVAR A MINÚSCULAS TODOS LOS CAMPOS DE TEXTO, SIN EXCEPCIÓN
platforms['show_id'] = platforms['show_id'].str.lower()
platforms['type'] = platforms['type'].str.lower()
platforms['title'] = platforms['title'].str.lower()
platforms['director'] = platforms['director'].str.lower()
platforms['cast'] = platforms['cast'].str.lower()
platforms['country'] = platforms['country'].str.lower()
platforms['class'] = platforms['class'].str.lower()
platforms['duration'] = platforms['duration'].str.lower()
platforms['listed_in'] = platforms['listed_in'].str.lower()
platforms['description'] = platforms['description'].str.lower()

In [21]:
#CONVERTIR CAMPO duration EN DOS CAMPOS (duration_int(INT) Y duration_type(STRING))
platforms[['duration_int', 'duration_type']] = platforms['duration'].str.split(' ', 1, expand=True)

C:\Users\Pedro Hoyos\AppData\Local\Temp\ipykernel_7892\1059396421.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  platforms[['duration_int', 'duration_type']] = platforms['duration'].str.split(' ', 1, expand=True)


In [23]:
#ELIMINAR COLUMNA duration LA CUAL FUE DIVIDIDA
platforms.drop(columns = ['duration'], inplace=True)

In [24]:
#REINDEXAR COLUMNAS (id) A COLUMNA INICIAL
platforms = platforms.reindex(columns=['show_id','type','title','listed_in','score','director','cast','country','date_added','release_year','class','duration_int','duration_type','description'])

In [26]:
#GUARDAR ARCHIVO CON MODIFICACIONES REALIZADAS
platforms.to_csv("platforms.csv", sep=';')

In [27]:
platforms.head()

,show_id,type,title,listed_in,score,director,cast,country,date_added,release_year,class,duration_int,duration_type,description
0,as1,movie,the grand seduction,"comedy, drama",3.467131,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,a small fishing village must procure a local d...
1,as2,movie,take care good night,"drama, international",3.548682,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110,min,a metro family decides to fight a cyber crimin...
2,as3,movie,secrets of deception,"action, drama, suspense",3.500000,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74,min,after a man discovers his wife is cheating on ...
3,as4,movie,pink: staying true,documentary,3.538055,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,69,min,"pink breaks the mold once again, bringing her ..."
4,as5,movie,monster maker,"drama, fantasy",3.478992,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,45,min,teenage matt banting wants to work with a famo...


In [29]:
print(platforms['score'].min())
print(platforms['score'].max())
print(platforms['score'].mean())

3.3364779874213837
3.7245119305856833
3.5334423547247473


In [30]:
platforms.shape

(22998, 14)

# Funciones requeridas para las consultas solicitadas

Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN. (la función debe llamarse get_max_duration(year, platform, duration_type))

In [1]:
print("Hola")

Hola


In [2]:
import pandas as pd
platforms = pd.read_csv('platforms.csv', sep=(';'))

In [3]:
def get_max_duration(year, platform, duration_type):
    film_time = []
    film_name = []
    for k, l in enumerate (platforms['release_year']):
        if l == year:
            if platforms['show_id'][k][0] == platform:
                if platforms['duration_type'][k] == duration_type:
                    film_time.append(int(platforms['duration_int'][k]))
                    film_name.append(k)
                    maximo = max(film_time)
                    film = film_time.index(maximo)
                    nom_sho = platforms['title'][film_name[film]]
    return nom_sho

In [62]:
print(get_max_duration(2018, 'd', 'min'))

marvel studios' avengers: infinity war


Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año (la función debe llamarse get_score_count(platform, scored, year)

In [5]:
def get_score_count(platform, scored, year):
    max_score = 0
    for m, n in enumerate(platforms['release_year']):
        if n == year:
            if platforms['show_id'][m][0] == platform:
                if platforms['score'][m] > scored:
                    max_score += 1
    return max_score

In [63]:
print(get_score_count('d', 3.5, 2018))

56


Cantidad de películas por plataforma con filtro de PLATAFORMA. (La función debe llamarse get_count_platform(platform))

In [64]:
def get_count_platform(platform):
    pltf_films = 0
    for o, p in enumerate(platforms['show_id']):
        if platforms['show_id'][o][0] == platform:
            pltf_films +=1
    return pltf_films

In [65]:
print(get_count_platform('a'))
print(get_count_platform('d'))
print(get_count_platform('h'))
print(get_count_platform('n'))

9668
1450
3073
8807


Actor que más se repite según plataforma y año. (La función debe llamarse get_actor(platform, year)

In [57]:
def get_actor(platform, year):
    actors1 = []
    actors2 = []
    conta = 0
    if platform == "h":
        return("No está disponible la información de reparto en el momento. Por favor discúlpenos")
    for q, r in enumerate (platforms['release_year']):
        if r == year:
            if platforms['show_id'][q][0] == platform:
                actors1.append(platforms['cast'][q])
    for s, t in enumerate (actors1):
        if type(t) == str:
            actors1[s] = t.split(", ")
    for t in actors1:
        if type(t) == list:
            actors2.append(t)
    actors3 = [elem for sub in actors2 for elem in sub]
    for u, v in enumerate (actors3):
        if actors3.count(v) > conta:
            conta = actors3.count(v)
            d = actors3[u]
    return d

In [67]:
print(get_actor('n', 2015))

james franco
